# Establishing a Connection - Bob
## Role: Invitee

If you want to start at Part 1 of this tutorial series go [here](http://localhost:8888/lab/tree/1%20Basic%20Concepts%20and%201st%20Connection/Part%201%20-%20Self-Sovereign%20Identity.ipynb).

In this notebook we'll be responding to an invitation from Alice through our Agents. This notebook has the following phases:

<b>Begin with the [Alice notebook](http://localhost:8888/lab/tree/1%20Basic%20Concepts%20and%201st%20Connection/Part%203%20-%20Establishing%20a%20Connection.ipynb)</b>


6. Instatiate the controller for our Agent
7. Set up a message listener running as a service on our controller
8. Paste the invitation from the Alice notebook into the invitation variable here
9. Accept the invitation

<b>Carry on at the Alice side</b>
    
14. Check if connection active


### 6. Instatiate the controller for our Agent

In [ ]:
%autoawait
import time
import asyncio
from aries_basic_controller.aries_controller import AriesAgentController
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8052
WEBHOOK_BASE = ""
ADMIN_URL = "http://bob-agent:8051"

agent_controller = AriesAgentController(webhook_host=WEBHOOK_HOST, webhook_port=WEBHOOK_PORT,
                                       webhook_base=WEBHOOK_BASE, admin_url=ADMIN_URL)

### 7. Listen for webhooks and register default listeners

Everytime a webhook is received from the agent, the controller reemits the hook using PyPubSub. The default listeners are used to update state and print logs.


In [ ]:

loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())

def connection_handler(payload):
    print("Connection Handler Called")
    connection_id = payload["connection_id"]
    state = payload["state"]
    print(f"Connection {connection_id} in State {state}")
    
connection_listener = {
    "handler": connection_handler,
    "topic": "connections"
}

agent_controller.register_listeners([connection_listener], defaults=True)

### 8. Paste the invitation from the Alice notebook into the invitation variable here;

In [ ]:
#Paste in invitation from Alice agent
invitation = {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation', '@id': 'e755dc99-9f61-4ab8-a59c-3f449a0287c3', 'label': 'Alice', 'recipientKeys': ['7rh9AtsmZg2QXVgePCUgsGUSn4VVAcxeBeadSMVmwkrb'], 'serviceEndpoint': 'http://172.17.0.1:8020'}

### 9. Accept the invitation, then move to Alice's notebook

In [ ]:
# Receive Invitation
response = await agent_controller.connections.accept_connection(invitation)
# Print out accepted Invite and Alice's connection ID
print("Connection", response)
alice_id = response["connection_id"]


### 13. Check if connection state is active

It should be if you send a trust ping from Alice's side

In [ ]:
# Print connection list
connection = await agent_controller.connections.get_connection(alice_id)
print("Alice AGENT CONNECTION")
print(connection)
print("State:", connection["state"])

## Optional: Remove connection

In [ ]:
remove = await agent_controller.connections.remove_connection(alice_id)
print(remove)

In [ ]:
response = await agent_controller.connections.get_connections()
print(response)

## End of Tutorial

#### Terminate Controller & Stop Webhook Server

**Note: You will need to run this command when combining this example with others such as Issuer**

In [ ]:
response = await agent_controller.terminate()
print(response)